In [ ]:
import pathlib
import polars as pl

datadir = pathlib.Path.cwd().parent / "data"

pl_hotel       = pl.read_parquet(datadir / "hotel.parquet")
pl_customer    = pl.read_parquet(datadir / "customer.parquet")
pl_reservation = pl.read_parquet(datadir / "reservation.parquet")
pl_campaign    = pl.read_parquet(datadir / "campaign.parquet")

# 15章 演習問題
## 15-1 多次元分析の前処理
### Q: 予約履歴の多次元分析のためのキューブ作成


In [ ]:
(
    # （1） reservationから必要なデータを抽出
    pl_reservation
    # （1）-1 キャンセルを除外
    .filter(pl.col("status") != "canceled")
    # （1）-2 2019年のデータを抽出
    .filter(pl.col("checkout_date").dt.year() == 2019)
    .with_columns(
        # （1）-3 checkout_dateから年と月を抽出
        checkout_year=pl.col("checkout_date").dt.year(),
        checkout_month=pl.col("checkout_date").dt.month(),
        # （1）-4 campaignと結合するために予約日時を丸めて予約日の列を作成
        reserve_date=pl.col("reserved_at").dt.date(),
    )
    # （2） customerと結合
    .join(
        pl_customer
        .select([
            "customer_id",
            "sex",
            # （2）-1 年齢を10刻みでカテゴリ化、60以上は一つのカテゴリにまとめる
            pl.when(pl.col("age") >= 60).then(6)
            .otherwise(pl.col("age") / 10).cast(int)
            .alias("age_cat")
        ]),
        on="customer_id", how="left"
    )
    # （3） campaignと結合
    .join(
        pl_campaign
        .select([
            "campaign_name",
            # （3）-1 キャンペーン期間内の日をすべて列挙
            pl.date_ranges(
                pl.col("starts_at").dt.date(),
                pl.col("ends_at").dt.date(),
                "1d"
            )
            .alias("reserve_date"),
        ])
        .explode("reserve_date"),
        on="reserve_date", how="left"
    )
    # （3）-2 結合用のカラムを削除
    .drop("reserve_date")
    # （4） ディメンションをキーとしてreservationを集約
    .group_by(["checkout_year", "checkout_month", "hotel_id", "sex", "age_cat",
              "campaign_name"])
    .agg([
        pl.col("total_price").sum().alias("sales"),
        pl.col("reservation_id").count().alias("reservation_cnt"),
        pl.col("length_of_stay").mean().alias("length_of_stay_avg"),
        pl.col("people_num").mean().alias("people_num_avg"),
    ])
    # （5） 集約データにhotelを結合
    .join(
        pl_hotel
        .select([
            "hotel_id", "hotel_name", "hotel_type", "address_prefecture",
            "address_town",
            # （5）-1 unit_priceをカテゴリ化
            pl.when(pl.col("unit_price") < 5000).then(0)
                .when(pl.col("unit_price") < 10000).then(5000)
                .when(pl.col("unit_price") < 20000).then(10000)
                .when(pl.col("unit_price") < 30000).then(20000)
                .otherwise(30000).alias("unit_price_range")
        ]),
        on="hotel_id", how="left"
    )
)